In [3]:
import numpy as np
import os
import matplotlib
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt


import pandas as pd


from scipy import integrate
import re



from qetpy.core._fitting import *

matplotlib.rc('font', size=16)          # controls default text sizes
matplotlib.rc('axes', titlesize=16)     # fontsize of the axes title
matplotlib.rc('axes', labelsize=16)    # fontsize of the x and y labels
matplotlib.rc('xtick', labelsize=14)    # fontsize of the tick labels
matplotlib.rc('ytick', labelsize=14)    # fontsize of the tick labels
matplotlib.rc('legend', fontsize=16)    # legend fontsize
matplotlib.rc('figure', titlesize=16)  # fontsize of the figure title
matplotlib.rcParams['figure.dpi'] = 100
matplotlib.rc('figure.subplot', hspace=  0.4)

def create_template(traces):
    template = np.mean(traces, axis = 0)
    template_norm= template/np.linalg.norm(template)
    return template, template_norm

def OF_calc(signal, template_norm, psd_noise, fs, noisevar, recordlength):
    amp, t0, chi2, fit_amp_min, data_amp_min= [], [], [], [], []
    chi2_sav, chi2_my, chi2_array_varnoise = [], [], []
    integrated_fit_amp, integrated_raw_amp = [], []
    for trace in np.arange(len(signal)):
        OF = OptimumFilter(signal[trace], template_norm, psd_noise, fs) # initialize the OptimumFilter class
        amp_withdelay, t0_withdelay, chi2_withdelay= OF.ofamp_withdelay() # t0_withdelay : time shift calculated for the pulse (in s).
        amp.append(amp_withdelay)
        t0.append(t0_withdelay)
        chi2.append(chi2_withdelay)
        fit_amp = amp_withdelay*np.roll(template_norm, int(t0_withdelay*fs))
        fit_amp_min.append(np.min(fit_amp )) 
        data_amp_min.append(np.min(signal[trace]))
        integrated_fit_amp.append(-1* integrate.trapz(y = fit_amp, dx = 2e-9 ) )
        integrated_raw_amp.append(-1* integrate.trapz(y = signal[trace], dx = 2e-9 ) )

        chi2_sav.append((abs(signal[trace] - fit_amp).sum())/recordlength)
        chi2_my.append((((signal[trace] - fit_amp)**2/abs(fit_amp)).sum())/recordlength )
        chi2_array_varnoise.append ( (((signal[trace] - fit_amp)**2/noisevar).sum())/recordlength)

    return amp, t0, chi2, fit_amp_min, data_amp_min, chi2_sav, chi2_my, chi2_array_varnoise, integrated_fit_amp, integrated_raw_amp


signal = np.loadtxt('test_wavedump/wavedumpdata/wavedump_Tl_sample.txt')
signal = signal-np.expand_dims(signal[:,100:2000].mean(axis=1), axis=1)
signal = signal [0:2]

template =create_template(np.load('test_wavedump/wavedumpdata/Tl_template_pulses.npy'))[1]


amp_withdelay, t0_withdelay, chi2_withdelay, fit_amp_min, data_amp_min, chi2_sav, chi2_my, chi2_varnoise, integrated_fit_amp, integrated_raw_amp= OF_calc(signal= signal, 
                                                                              template_norm = create_template(np.load('test_wavedump/wavedumpdata/Tl_template_pulses.npy'))[1],
                                                                              psd_noise = np.load('test_wavedump/saved_outputs/psd_noise_Tl.npy')[1],
                                                                              fs = 1/2e-9, 
                                                                              noisevar = 7.029123062695451,
                                                                              recordlength=15000)

33450669.226085205 33333.33333333333
() (15000,) (15000,) (15000,) (15000,)
in bestind decision
finishing bestind
finishing OF..
30449831.26720471 33333.33333333333
() (15000,) (15000,) (15000,) (15000,)
in bestind decision
finishing bestind
finishing OF..


In [ ]:
print(amp_withdelay, t0_withdelay)

In [ ]:
print (min(template))

In [ ]:
plt.plot(np.arange(15000)/2e3, signal[0])
plt.plot(np.arange(15000)/2e3, 10000*template)